In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_category = 'Sarees'

In [ ]:
path_eff_b6 = "../../Models/eff_net_b6_fillna_eff_net_Sarees.pth"
path_eff_b5 = "../../Models/eff_net_fillna_eff_net_Sarees.pth"

In [ ]:
train_df_Men_Tshirts = pd.read_csv('../../Preprocessor-FillNA/output/train_fillna_sarees_effnet_b5_4epochs.csv')
train_df_Men_Tshirts

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10,image_path
0,7432,Sarees,10,same as saree,woven design,small border,multicolor,party,jacquard,woven design,zari woven,applique,no,/kaggle/input/visual-taxonomy/train_images/007...
1,7433,Sarees,10,same as saree,zari,small border,cream,traditional,jacquard,woven design,zari woven,elephant,no,/kaggle/input/visual-taxonomy/train_images/007...
2,7434,Sarees,10,same as saree,zari,small border,white,party,jacquard,woven design,zari woven,floral,no,/kaggle/input/visual-taxonomy/train_images/007...
3,7435,Sarees,10,same as saree,woven design,big border,default,traditional,jacquard,same as saree,zari woven,ethnic motif,no,/kaggle/input/visual-taxonomy/train_images/007...
4,7436,Sarees,10,solid,no border,small border,multicolor,daily,default,same as saree,printed,default,yes,/kaggle/input/visual-taxonomy/train_images/007...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18341,25773,Sarees,10,same as saree,zari,small border,cream,party,jacquard,woven design,zari woven,peacock,no,/kaggle/input/visual-taxonomy/train_images/025...
18342,25774,Sarees,10,same as saree,no border,no border,default,party,default,zari woven,zari woven,default,no,/kaggle/input/visual-taxonomy/train_images/025...
18343,25775,Sarees,10,same as saree,zari,small border,cream,party,jacquard,woven design,zari woven,peacock,no,/kaggle/input/visual-taxonomy/train_images/025...
18344,25776,Sarees,10,same as saree,temple border,small border,cream,party,jacquard,woven design,zari woven,peacock,no,/kaggle/input/visual-taxonomy/train_images/025...


In [5]:
attr_columns = train_df_Men_Tshirts.filter(like='attr_').columns.to_list() # Adjust if more attributes exist
print(attr_columns)

['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']


In [6]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder for each attribute column
label_encoders = {}
for column in attr_columns:
    le = LabelEncoder()
    train_df_Men_Tshirts[column] = le.fit_transform(train_df_Men_Tshirts[column])
    label_encoders[column] = le  # Store the encoder for inverse transformation later if needed

# Check the updated DataFrame
# train_df_Men_Tshirts = train_df_Men_Tshirts.head()

In [7]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df1, val_df1 = train_test_split(train_df_Men_Tshirts, test_size=0.15, random_state=42)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test  # Flag to indicate if it's test set without labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.img_dir + self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:  # For test set, just return the image without labels
            return image
        
        # For train/validation set, return image and labels
        labels = self.dataframe.iloc[idx][attr_columns].values
        labels = labels.astype(np.int64)  # Ensure labels are integers
        labels = torch.tensor(labels, dtype=torch.long)
        
        return image, labels
    
    
# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Apply Color Jitter
    transforms.RandomHorizontalFlip(),  # Apply Horizontal Flip with 50% probability
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
    
train_dataset = CustomDataset(dataframe=train_df1, img_dir='', transform=transform)
val_dataset = CustomDataset(dataframe=val_df1, img_dir='', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class MultiOutputModel1(nn.Module):
    def __init__(self, num_classes):
        super(MultiOutputModel1, self).__init__()
        # Use MobileNetV2 as the base model
        self.base_model = models.efficientnet_b6(pretrained=True)
        self.base_model.classifier = nn.Identity()  # Remove original classification layer
        
        # Add an adaptive pooling layer to make sure output is flat
        self.pooling = nn.AdaptiveAvgPool2d((1, 1))  # Convert 2D output to 1D
        
        # Dynamically create a fully connected layer for each attribute
        self.output_layers = nn.ModuleDict()
        for attr, n_classes in num_classes.items():
            self.output_layers[attr] = nn.Linear(2304, n_classes)  # Adjust input to 1280 for MobileNetV2

    def forward(self, x):
        x = self.base_model.features(x)  # Extract features
        x = self.pooling(x)  # Adaptive pool to (1, 1) shape
        x = torch.flatten(x, 1)  # Flatten the output to (batch_size, 1280)
        
        outputs = {}
        # Dynamically compute output for each attribute
        for attr, layer in self.output_layers.items():
            outputs[attr] = layer(x)
        
        return list(outputs.values())

# Example usage
num_classes = {}
for key in attr_columns:
    num_classes[key] = train_df_Men_Tshirts[key].nunique()

model_eff_b6 = MultiOutputModel1(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_eff_b6.parameters(), lr=0.0001)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B6_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B6_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b6_lukemelas-24a108a5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b6_lukemelas-24a108a5.pth
100%|██████████| 165M/165M [00:00<00:00, 183MB/s]  


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class MultiOutputModel2(nn.Module):
    def __init__(self, num_classes):
        super(MultiOutputModel2, self).__init__()
        # Use MobileNetV2 as the base model
        self.base_model = models.efficientnet_b5(pretrained=True)
        self.base_model.classifier = nn.Identity()  # Remove original classification layer
        
        # Add an adaptive pooling layer to make sure output is flat
        self.pooling = nn.AdaptiveAvgPool2d((1, 1))  # Convert 2D output to 1D
        
        # Dynamically create a fully connected layer for each attribute
        self.output_layers = nn.ModuleDict()
        for attr, n_classes in num_classes.items():
            self.output_layers[attr] = nn.Linear(2048, n_classes)  # Adjust input to 1280 for MobileNetV2

    def forward(self, x):
        x = self.base_model.features(x)  # Extract features
        x = self.pooling(x)  # Adaptive pool to (1, 1) shape
        x = torch.flatten(x, 1)  # Flatten the output to (batch_size, 1280)
        
        outputs = {}
        # Dynamically compute output for each attribute
        for attr, layer in self.output_layers.items():
            outputs[attr] = layer(x)
        
        return list(outputs.values())

# Example usage
num_classes = {}
for key in attr_columns:
    num_classes[key] = train_df_Men_Tshirts[key].nunique()

model_eff_b5 = MultiOutputModel2(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_eff_b5.parameters(), lr=0.0001)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B5_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b5_lukemelas-1a07897c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b5_lukemelas-1a07897c.pth
100%|██████████| 117M/117M [00:00<00:00, 202MB/s]  


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_eff_b6 = model_eff_b6.to(device)
model_eff_b5 =  model_eff_b5.to(device)
# Training loop
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, save_path=f"eff_net_b6_fillna_eff_net_{model_category}.pth"):
    scheduler = optim.lr_scheduler.StepLR(optimizer = optimizer , step_size=5, gamma=0.5)
    best_val_loss = float('inf')
    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device
            optimizer.zero_grad()
            outputs = model(images)
            loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        scheduler.step()
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}')

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device
                outputs = model(images)
                loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        print(f'Epoch {epoch+1}, Val Loss: {val_loss/len(val_loader)}')
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved with Val Loss: {best_val_loss:.4f} at Epoch {epoch+1}")

# Run training
# train(model, train_loader, val_loader, criterion, optimizer, num_epochs=7)

In [13]:
torch.manual_seed(42)
model_eff_b6.load_state_dict(torch.load(f"{path_eff_b6}"))

/tmp/ipykernel_30/563325272.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_eff_b6.load_state_dict(torch.load(f"{path_eff_b6}"))


<All keys matched successfully>

In [14]:
torch.manual_seed(42)
model_eff_b5.load_state_dict(torch.load(f"{path_eff_b5}"))

/tmp/ipykernel_30/399784212.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_eff_b5.load_state_dict(torch.load(f"{path_eff_b5}"))


<All keys matched successfully>

In [15]:
torch.manual_seed(42)
model_eff_b6.eval()

MultiOutputModel1(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 56, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(56, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(56, 56, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=56, bias=False)
              (1): BatchNorm2d(56, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(56, 14, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(14, 56, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
         

In [16]:
torch.manual_seed(42)
model_eff_b5.eval()

MultiOutputModel2(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
              (1): BatchNorm2d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
         

In [ ]:
test_df = pd.read_csv('../../Dataset/test.csv')
test_df

,id,Category
0,0,Men Tshirts
1,1,Men Tshirts
2,2,Men Tshirts
3,3,Men Tshirts
4,4,Men Tshirts
...,...,...
30200,30484,Women Tops & Tunics
30201,30485,Women Tops & Tunics
30202,30486,Women Tops & Tunics
30203,30487,Women Tops & Tunics


In [ ]:
test_df_Men_Tshirts = test_df[test_df['Category'] == model_category]
test_df_Men_Tshirts['len'] = 10

def format_image_path_test(row):
    return f"../../Dataset/test_images/{str(row['id']).zfill(6)}.jpg"

test_df_Men_Tshirts['image_path'] = test_df_Men_Tshirts.apply(format_image_path_test, axis=1)
test_df_Men_Tshirts

/tmp/ipykernel_30/2828864867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts['len'] = 10
/tmp/ipykernel_30/2828864867.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts['image_path'] = test_df_Men_Tshirts.apply(format_image_path_test, axis=1)


,id,Category,len,image_path
3787,3787,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...
3788,3788,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...
3789,3789,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...
3790,3790,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...
3791,3791,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...
...,...,...,...,...
10884,11150,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0111...
10885,11151,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0111...
10886,11152,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0111...
10887,11153,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0111...


In [20]:
# Test dataset without labels
test_dataset = CustomDataset(dataframe=test_df_Men_Tshirts, img_dir='', transform=transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [21]:
len(test_loader)

222

In [22]:
from tqdm import tqdm  # Use console version of tqdm

predictions = []

with torch.no_grad():
    for images in tqdm(test_loader):  # This will display in the console
        images = images.cuda() if torch.cuda.is_available() else images

        # Forward pass through both models
        torch.manual_seed(42)
        outputs1 = model_eff_b6(images)
        outputs2 = model_eff_b5(images)

        # Initialize a list to hold blended predictions for the batch
        batch_preds = []

        # Loop through the outputs and blend predictions for each attribute
        for out1, out2 in zip(outputs1, outputs2):
            # Blend logits by averaging
            blended_output = (out1 + out2)/2

            # Get the predicted classes from the blended output
            _, pred = torch.max(blended_output, 1)
            batch_preds.append(pred.cpu().numpy())  # Move to CPU and convert to numpy

        # Stack predictions along a new dimension and add to predictions list
        predictions.append(np.stack(batch_preds, axis=1))  # Shape: (batch_size, num_attributes)

# Combine all predictions into a single array
predictions = np.concatenate(predictions, axis=0)

# Display final predictions
print(predictions)  # This will be an array with shape (num_samples, num_attributes)


100%|██████████| 222/222 [03:33<00:00,  1.04it/s]

[[0 0 2 ... 0 3 0]
 [1 4 0 ... 4 5 0]
 [3 5 0 ... 2 8 0]
 ...
 [2 4 0 ... 4 3 0]
 [2 4 0 ... 4 3 0]
 [2 5 2 ... 4 7 0]]


In [23]:
# Assuming predictions is a numpy array of shape (N, 5)
# Add new columns attr_1 to attr_10 to test_df
for i in range(1, 11):
    test_df_Men_Tshirts[f'attr_{i}'] = np.nan 

# Assign predictions to attr_1 to attr_5
test_df_Men_Tshirts[attr_columns] = predictions

# Optionally save the updated test_df to CSV
# test_df.to_csv('test_predictions_with_attrs.csv', index=False)

# Display the first few rows of the updated DataFrame
test_df_Men_Tshirts.head()

/tmp/ipykernel_30/1201814413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[f'attr_{i}'] = np.nan
/tmp/ipykernel_30/1201814413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[f'attr_{i}'] = np.nan
/tmp/ipykernel_30/1201814413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

,id,Category,len,image_path,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
3787,3787,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...,0,0,2,1,1,0,1,0,3,0
3788,3788,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...,1,4,0,3,0,1,3,4,5,0
3789,3789,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...,3,5,0,1,1,1,3,2,8,0
3790,3790,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...,1,4,0,1,2,1,3,4,5,0
3791,3791,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...,0,4,1,3,2,0,3,4,3,0


In [24]:
# Inverse transform predictions for each attribute using the stored label encoders
for attr in attr_columns:
    # Inverse transform using the corresponding label encoder
    test_df_Men_Tshirts[attr] = label_encoders[attr].inverse_transform(test_df_Men_Tshirts[attr].astype(int))

# Check the updated DataFrame¯ with original attribute names instead of encoded numbers
test_df_Men_Tshirts

/tmp/ipykernel_30/688606145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[attr] = label_encoders[attr].inverse_transform(test_df_Men_Tshirts[attr].astype(int))
/tmp/ipykernel_30/688606145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[attr] = label_encoders[attr].inverse_transform(test_df_Men_Tshirts[attr].astype(int))
/tmp/ipykernel_30/688606145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,id,Category,len,image_path,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
3787,3787,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...,default,default,small border,default,party,default,same as saree,default,default,no
3788,3788,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...,same as border,woven design,big border,multicolor,daily,jacquard,zari woven,zari woven,ethnic motif,no
3789,3789,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...,solid,zari,big border,default,party,jacquard,zari woven,solid,solid,no
3790,3790,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...,same as border,woven design,big border,default,traditional,jacquard,zari woven,zari woven,ethnic motif,no
3791,3791,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0037...,default,woven design,no border,multicolor,traditional,default,zari woven,zari woven,default,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10884,11150,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0111...,default,woven design,no border,multicolor,traditional,default,zari woven,zari woven,default,no
10885,11151,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0111...,same as border,zari,big border,default,party,default,same as saree,zari woven,default,no
10886,11152,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0111...,same as saree,woven design,big border,default,daily,jacquard,zari woven,zari woven,default,no
10887,11153,Sarees,10,/kaggle/input/visual-taxonomy/test_images/0111...,same as saree,woven design,big border,default,traditional,default,same as saree,zari woven,default,no


In [25]:
# Get value counts for each specified column
columns_of_interest = attr_columns

for column in columns_of_interest:
    print(f"Value counts for {column}:\n")
    print(test_df_Men_Tshirts[column].value_counts(dropna=False))  # Including NaN values
    print("\n" + "-"*50 + "\n")

Value counts for attr_1:

attr_1
same as saree     4131
same as border    1469
default           1063
solid              439
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_2:

attr_2
woven design     2502
zari             2384
default          1406
temple border     465
solid             201
no border         144
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_3:

attr_3
small border    3304
big border      3265
no border        533
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_4:

attr_4
default       3189
cream         1864
multicolor    1478
navy blue      134
yellow         122
pink           122
white          115
green           78
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_5:

attr_5
party          4406
traditional    1598
wedding         564
daily           534


In [ ]:
sub_df = pd.read_csv('../../Dataset/sample_submission.csv')
sub_df

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,47001,Women Tshirts,8,maroon,regular,crop,printed,typography,short sleeves,regular sleeves,default,nu,nu
1,16431,Sarees,10,same as saree\n,temple border,small border,cream,party,jacquard,woven design,zari woven,ethnic motif,no
2,55700,Women Tops & Tunics,10,white,fitted,regular,round neck,casual,solid,solid,short sleeves,regular sleeves,knitted
3,15698,Sarees,10,same as saree\n,zari,small border,white,party,jacquard,woven design\n,zari woven,floral,no
4,30330,Kurtis,9,yellow,a-line,knee length\n,daily,net,default,solid,three-quarter sleeves,regular,nu


In [27]:
test_df_Men_Tshirts = test_df_Men_Tshirts.fillna('dummy')

In [ ]:
sub_df_Men_Tshirts = test_df_Men_Tshirts.drop('image_path', axis = 1)
sub_df_Men_Tshirts.to_csv(f"output/sub_df_{model_category}_effnet_0.5_b6_0.5_b5_blending.csv", index = False)
sub_df_Men_Tshirts.to_csv(f"sub_df_{model_category}_effnet_0.5_b6_0.5_b5_blending.csv", index = False)
sub_df_Men_Tshirts

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
3787,3787,Sarees,10,default,default,small border,default,party,default,same as saree,default,default,no
3788,3788,Sarees,10,same as border,woven design,big border,multicolor,daily,jacquard,zari woven,zari woven,ethnic motif,no
3789,3789,Sarees,10,solid,zari,big border,default,party,jacquard,zari woven,solid,solid,no
3790,3790,Sarees,10,same as border,woven design,big border,default,traditional,jacquard,zari woven,zari woven,ethnic motif,no
3791,3791,Sarees,10,default,woven design,no border,multicolor,traditional,default,zari woven,zari woven,default,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10884,11150,Sarees,10,default,woven design,no border,multicolor,traditional,default,zari woven,zari woven,default,no
10885,11151,Sarees,10,same as border,zari,big border,default,party,default,same as saree,zari woven,default,no
10886,11152,Sarees,10,same as saree,woven design,big border,default,daily,jacquard,zari woven,zari woven,default,no
10887,11153,Sarees,10,same as saree,woven design,big border,default,traditional,default,same as saree,zari woven,default,no
